In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.model_selection import train_test_split

## Veri Keşfi ve Görselleştirme

In [ ]:
veri = pd.read_csv('../input/spotify-dataset-19212020-160k-tracks/data.csv')


In [ ]:
veri.describe()

In [ ]:
veri.info()

In [ ]:
veri.head()

In [ ]:
veri.tail()

In [ ]:
veri.shape

In [ ]:
import matplotlib.pyplot as plt
veri.hist(figsize=(15, 15), color='red')
plt.show()

In [ ]:
veri.corr()

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(veri.corr(), annot=True)

#Korelasyon grafiği incelendiğinde yıl, dans edilebilirlik, enerji,
#ses yüksekliği ve tempo değişkenleri popülerliği tahmin etmede önemli özelliklerdir.

In [ ]:
plt.plot(veri.loudness, veri.energy, 'ro', color='green')

## Ön İşleme

In [ ]:
veri.isnull().sum() #eksik değerlerin kontrolü

In [ ]:
from collections import Counter
def uc_degerler(df,ozellikler):
    uc_degerler_index = []
    
    for c in ozellikler:
        # 1.inci çeyreklik 
        Q1 = np.percentile(veri[c],25)
        # 3.üncü çeyreklik
        Q3 = np.percentile(veri[c],75)
        # Interquartile Range (IQR) yöntemi
        IQR = Q3 - Q1
        alt_sinir = Q1-1.5*IQR #alt sınır değeri
        ust_sinir = Q3+1.5*IQR #üst sınır değeri
        #aykırı değerler ve bu değerlerin endekslerinin tespit edilmesi
        uc_deger_listesi = veri[(veri[c] < alt_sinir) | (veri[c] > ust_sinir)].index #alt ve üst sınırların altında kalan değerlerin indexi
        # store indeces
        uc_degerler_index.extend(uc_deger_listesi)
    
    uc_degerler_index = Counter(uc_degerler_index)
    multiple_outliers = list(i for i, v in uc_degerler_index.items() if v > 2) 
    
    return multiple_outliers

In [ ]:
veri.loc[uc_degerler(veri,["popularity","explicit","year","duration_ms","danceability","energy","instrumentalness","liveness","loudness","speechiness","valence","key","mode","tempo"])]
#Çıkan sonuçlar incelendiğinde 2704 adet verinin belirlenen değerler dışında olduğu belirlenmiştir.

In [ ]:
sns.boxplot(x=veri['danceability'])

In [ ]:
sns.boxplot(x=veri['loudness'])

In [ ]:

#Uç verilerin veri setinden çıkarılması
veri= veri.drop(uc_degerler(veri,["popularity","explicit","year","duration_ms","danceability","energy","instrumentalness","liveness","loudness","speechiness","valence","key","mode","tempo"]))


In [ ]:
#Veri setinin uç değerler çıkarıldıktan sonraki yeni boyutu
veri.shape

In [ ]:
#Popularity değişkeninde yeni bir öznitelik oluşturma 
veri["popüler"]= [ 1 if i>=70 else 0 for i in veri.popularity ]
veri["popüler"].value_counts()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

tahmin_input = ['year', 'danceability', 'energy', 'loudness', 'tempo']# popülerliği tahmin etmede kullanılacqk giriş verileri
data = veri.copy()
onemli_ozellikler = veri[tahmin_input]

normalize = StandardScaler()
normalize.fit(onemli_ozellikler)
onemli_ozellikler = normalize.transform(onemli_ozellikler)

print(onemli_ozellikler.shape) 



## Model Eğitimi

In [ ]:
y = veri["popüler"].values

from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(onemli_ozellikler, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
import seaborn as sns
from sklearn.metrics import classification_report


rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
RandomForest_score=rf.score(X_test,y_test)
print("Training Accuracy :", rf.score(X_train, y_train))
print("Validation Accuracy :", rf.score(X_test, y_test))

print("F1 score :", f1_score(y_test, y_pred,average='macro'))
print(classification_report(y_test, y_pred))

# confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
knn_score=knn.score(X_test,y_test)
print("Training Accuracy :", knn.score(X_train, y_train))
print("Validation Accuracy :", knn.score(X_test, y_test))

# test veri setinin f1 puanının hesaplanması
print("f1 score :", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True)

In [ ]:
#kullanılan her iki algoritmanın sonuçları incelendiğinde Rastgele Orman Algoritmasının başarı oranı 
#daha yüksektir.
model_performances=pd.DataFrame({'Model':['RandomForestClassifier','K-NearestNeighbors'],
                                 'Accuracy':[RandomForest_score,knn_score]})
model_performances.sort_values(by = "Accuracy",ascending=False)
